In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision import transforms
from torchvision.datasets import MNIST

In [73]:
import pytorch_lightning as pl

In [52]:
from accuracy import accuracy

#### Parameter Configuration

In [60]:
root="./data"
log_dir = "./runs"
input_size = 28*28
out_size = 10
epochs = 20
batch_size = 64
lr = 1e-3
transform = transforms.Compose([
    transforms.ToTensor()
])


### Pure PyTorch

### Data

In [66]:
train = MNIST(
    root=root,
    train=True,
    transform=transform,
    download=True
)
val = MNIST(
    root=root,
    train=False,
    transform=transform,
    download=True
)
train_loader = DataLoader(
    dataset=train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=6,
)
val_loader = DataLoader(
    dataset=train,
    batch_size=32,
    shuffle=True,
    num_workers=6
)

### Model

In [67]:
class LogisticRegression(nn.Module):
    
    def __init__(self, input_size=28*28, n_classes=10):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(in_features=input_size, out_features=n_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [68]:
model = LogisticRegression(input_size=input_size, n_classes=out_size)

### Loss, Optimizer and Tensorboard writer

In [69]:
# optimizer, loss and tensorboard writer function
optimizer = Adam(params=model.parameters(), lr=lr)
criterian = nn.CrossEntropyLoss()
writer = SummaryWriter(log_dir=log_dir)

### Training

In [70]:
epoch_loss = []
epochs_acc = []
steps = 0
for epoch in range(epochs):
    
    epoch_loss_train = []
    epoch_acc_train = []
    epoch_loss_val = []
    epoch_acc_val = []
    
    for i, batch in enumerate(train_loader):
        
        steps += 1
        
        images, labels = batch
        images = images.view(-1, 28*28)
        outputs = model(images)
        
        # determine the loss and acc
        loss = criterian(outputs, labels)
        acc = accuracy(model=model, data=batch, loader=False)
        
        # backpropagate the loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # append the step loss and step acc
        epoch_loss_train.append(loss.item())
        epoch_acc_train.append(acc)
        
        if i%500==0:
            print(f'epoch  {epoch} |  step  {steps} | train_loss {loss.item()} | train_acc {acc}')
            writer.add_scalar("step-wise training loss", loss.item(), steps)
            writer.add_scalar("step-wise training acc", acc, steps)
        
    for i, batch in enumerate(val_loader):
        with torch.no_grad():
            images, labels = batch
            images = images.view(-1, 28*28)
            outputs = model(images)

            # determine the loss and acc
            loss = criterian(outputs, labels)
            acc = accuracy(model=model, data=batch, loader=False)

            # append the step loss and step acc
            epoch_loss_val.append(loss.item())
            epoch_acc_val.append(acc)
            
    t_loss = sum(epoch_loss_train)/len(epoch_loss_train)
    t_acc = sum(epoch_acc_train)/len(epoch_acc_train)
    
    v_loss = sum(epoch_loss_val)/len(epoch_loss_val)
    v_acc = sum(epoch_acc_val)/len(epoch_acc_val)
    
    writer.add_scalar("training loss ", t_loss, epoch)
    writer.add_scalar("validation loss", v_loss, epoch)
    writer.add_scalar("training acc", t_acc, epoch)
    writer.add_scalar("validation acc", v_acc, epoch)
    

    print(f'end of epoch {epoch} | train_loss {t_loss} | train_acc {t_acc} | val_loss {v_loss} | val_acc {v_acc}')

epoch  0 |  step  1 | train_loss 2.307408094406128 | train_acc 6.25
epoch  0 |  step  501 | train_loss 0.39344289898872375 | train_acc 85.9375
end of epoch 0 | train_loss 0.543136858673238 | train_acc 86.75039978678038 | val_loss 0.3460503515640895 | val_acc 90.58333333333333
epoch  1 |  step  939 | train_loss 0.4421619772911072 | train_acc 85.9375
epoch  1 |  step  1439 | train_loss 0.1482522338628769 | train_acc 100.0
end of epoch 1 | train_loss 0.3235077039916505 | train_acc 91.03144989339019 | val_loss 0.3024573971649011 | val_acc 91.62333333333333
epoch  2 |  step  1877 | train_loss 0.23878110945224762 | train_acc 93.75
epoch  2 |  step  2377 | train_loss 0.3309534788131714 | train_acc 92.1875
end of epoch 2 | train_loss 0.2953710081035903 | train_acc 91.74606876332622 | val_loss 0.2838791015545527 | val_acc 92.115
epoch  3 |  step  2815 | train_loss 0.3046310245990753 | train_acc 93.75
epoch  3 |  step  3315 | train_loss 0.41535767912864685 | train_acc 85.9375
end of epoch 3 | tr

In [71]:
!tensorboard --logdir=runs

/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/macab/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

## With PyTorch-Lightning

In [111]:
class LogisticRegression(pl.LightningModule):
    
    
    def __init__(self, in_features = 28*28, n_classes = 10):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(in_features=in_features, out_features=n_classes)
    
    def forward(self, x):
        out = self.linear(x.view(-1, 28*28))
        return out
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=lr)
    
    def accuracy(self, y, y_hat):
        correct = (y==torch.argmax(y_hat, 1)).sum().item()
        total = y.size(0)
        return correct/total
        
    def train_dataloader(self):
        dataset = MNIST(root=root, train=True, transform=transform, download=True)
        loader = DataLoader(dataset=dataset, shuffle=True, batch_size=batch_size, num_workers=6)
        return loader
    
    def val_dataloader(self):
        dataset = MNIST(root=root, train=False, transform=transform, download=True)
        loader = DataLoader(dataset=dataset, shuffle=False, batch_size=batch_size, num_workers=6)
        return loader
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
#         acc = self.accuracy(y=y, y_hat=y_hat)
        tensorboard_logs = {"loss":loss}
        return {"loss":loss, "log":tensorboard_logs}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
#         avg_acc = torch.stack([x['train_acc'] for x in outputs]).mean()
        tensorboard_logs = {"epoch_train_loss":avg_loss}
        return {"epoch_train_loss":avg_loss, "log":tensorboard_logs}
        
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
#         acc = self.accuracy(y=y, y_hat=y_hat)
        tensorboard_logs = {"val_loss":loss}
        return {"val_loss":loss, "log":tensorboard_logs}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {"epoch_val_loss":avg_loss}
        return {"epoch_val_loss":avg_loss, "log":tensorboard_logs}
        

In [112]:
model = LogisticRegression()

In [113]:
trainer = pl.Trainer(gpus=[0], max_epochs=20)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


In [114]:
trainer.fit(model)


  | Name   | Type   | Params
------------------------------
0 | linear | Linear | 7 K   


Epoch 1:  86%|████████▌ | 938/1095 [00:03<00:00, 252.13it/s, loss=0.371, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 2:   0%|          | 0/1095 [00:00<?, ?it/s, loss=0.371, v_num=0]            

/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: The metric you returned None must be a Torch.Tensor instance, checkpoint not saved HINT: what is the value of val_loss in validation_end()?
  warnings.warn(*args, **kwargs)
/home/macab/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: Can save best model only with val_loss available, skipping.
  warnings.warn(*args, **kwargs)


Epoch 2:  86%|████████▌ | 938/1095 [00:03<00:00, 288.87it/s, loss=0.284, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  86%|████████▌ | 938/1095 [00:03<00:00, 298.33it/s, loss=0.334, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  86%|████████▌ | 938/1095 [00:03<00:00, 311.88it/s, loss=0.297, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  86%|████████▌ | 938/1095 [00:03<00:00, 304.80it/s, loss=0.283, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  86%|████████▌ | 938/1095 [00:03<00:00, 302.14it/s, loss=0.344, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  86%|████████▌ | 938/1095 [00:02<00:00, 312.75it/s, loss=0.304, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 8:  86%|████████▌ | 938/1095 [00:03<00:00, 288.78it/s, loss=0.302, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 9:  86%|████████▌ | 938/1095 [00:03<00:00, 291.02it/s, loss=0.277, v_num=0] 
Validating: 0it [00:00, ?it/s]
Epoch 10:  86%|████████▌ | 938/1095 [00:03<00:00, 285.40it/s, loss=0.283, v_num=0]
Valida

1